In [26]:
import torchvision
import torch
from datasets import load_dataset,Dataset
import datasets
import numpy as np

Files already downloaded and verified
Files already downloaded and verified


In [49]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

class CIFAR10(torch.utils.data.Dataset):
    def collectImage(self, img):
        t = np.array(img)
        return t.transpose((2,0,1))
    def __init__(self, split="train"):
        if split == "train":
            dataset = torchvision.datasets.CIFAR10("../data/", download=True)
        else:
            dataset = torchvision.datasets.CIFAR10("../data/", download=True, train=False)
        self.data = torch.FloatTensor(np.stack([self.collectImage(i[0]) for i in dataset])/255).to(device)
        self.label = torch.LongTensor(np.array([i[1] for i in dataset], dtype=np.int64)).to(device)
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, i):
        return self.data[i], self.label[i]

In [50]:
dataset = CIFAR10()

Files already downloaded and verified


In [68]:
from transformers import ViTConfig, ViTModel
configuration = ViTConfig(hidden_size = 128,
                          num_hidden_layers = 2,
                          num_attention_heads = 2,
                         intermediate_size=128,
                         hidden_dropout_prob=0.1)
model = ViTModel(configuration).to(device)

In [69]:
model

ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 128, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-1): 2 x ViTLayer(
        (attention): ViTAttention(
          (attention): ViTSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=128, out_features=128, bias=True)
          (intermediate_act_fn): GELUActivation()
 

In [70]:
total_params = sum(p.numel() for p in model.parameters())
total_params

339712